<a href="https://colab.research.google.com/github/gmoriki/C4RA-Python-Tutorials/blob/main/0623/0623_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第7回C4RA勉強会
## DataFrameのデータ結合・マージ

---

## ★ 勉強会の基本方針

* 5月12日(金)から6月30日(金)までの計8回を予定しています。
* 目標は**pandas.DataFrame形式の基本的な操作をマスターすること**です。

* 環境はGoogle Colaboratory(通称Colab)を使用します。Googleアカウントをご用意ください。
  * 各々のローカル環境でも実行可能です。
* 一部、東京大学「Pythonプログラミング入門」の教材を共有しながら勉強会を進めます。
  * 勉強会では**東大教材**と呼びます
  * URL：https://utokyo-ipp.github.io/index.html

* Colab（Jupyter Notebook）の詳しい使い方について、勉強会の中で深入りはしません
  * Colab立ち上げのURL：https://colab.research.google.com/
  * 便利なショートカット：https://blog.kikagaku.co.jp/google-colaboratory-shortcut

* Pythonに関する不明点があれば森木(Pumble,Mail,Twitter等)までご連絡ください
* 勉強会の内容でなくてもOKです（Colabの使い方や環境構築なども）
---

## ■ データの結合

### DataFrameの結合: concat(), merge(), join()

Pythonのpandasライブラリには、DataFrame間で操作を行うための `join()`, `merge()`, `concat()` の3つのメソッドがあります。

#### `merge()`

`merge()`関数はSQLスタイルの結合を行います。この関数は、特定の列（または複数列）をキーとして2つのDataFrameを結合します。

デフォルトでは、指定したキーに基づく内部結合（INNER JOIN）が行われますが、`how`引数を指定することでLEFT、RIGHT、OUTER JOINを行うことも可能です。


#### `concat()`

`concat()`は、主にDataFrameやSeriesを軸に沿って連結するための関数です。連結はデフォルトで行（index）に対して行われますが、列（columns）に対して行うことも可能です。

同時に、新たなインデックスや階層的インデックスを作成することも可能です。


#### `join()`

`join()`は、一つのDataFrameを他のDataFrameのインデックスに合わせて結合するための関数です。`merge()`とは異なり、主にインデックスに基づいた操作が行われます。

ただし、`merge()`のように列に基づいた結合も可能です。

これらの関数を使うときは、どのような結合が必要かにより、適切な関数を選択することが重要です。

---

#### 1. merge()

`merge()` 関数は、一つ以上のキーを基にDataFrameを結合します。SQLのJOIN操作と類似しています。

**参考URL**：
https://towardsdatascience.com/how-to-merge-pandas-dataframes-35afe8b1497c

In [ ]:
import pandas as pd

data1 = {
    '個人番号': [24561, 78943, 98276, 13456, 68745, 23689, 87654, 45123, 34902, 50389],
    '個人名': ['田中', '佐藤', '鈴木', '高橋', '伊藤', '山田', '中村', '小林', '加藤', '吉田'],
    '所属大学': ['東京大学', '京都大学', '早稲田大学', '一橋大学', '慶應義塾大学', '名古屋大学', '北海道大学', '神戸大学', '大阪大学', '九州大学'],
    '年齢': [28, 24, 26, 30, 27, 29, 25, 31, 23, 28],
    '科研費獲得額': [1000000, 500000, 300000, 800000, 600000, 400000, 700000, 200000, 900000, 350000]
}

df_personal = pd.DataFrame(data1)

data2 = {
    '個人番号': [24561, 13456, 23689, 45123, 50389,99999],
    '研究エフォート率': [0.85, 0.92, 0.78, 0.75, 0.81,0.99],
    '職階': ['准教授', '助教授', '教授', '講師', '助手','教授']
}

df_research = pd.DataFrame(data2)

In [ ]:
df_personal

In [ ]:
df_research

以降、二つのデータフレーム`df_personal`と`df_research`を使用します。

それぞれのキーは「個人番号」です。

#### 1. merge()

`merge()` 関数は、一つ以上のキーを基にDataFrameを結合します。SQLのJOIN操作と類似しています。

https://towardsdatascience.com/how-to-merge-pandas-dataframes-35afe8b1497c

![pandas merge](./Howtomerge.webp)

内部結合

In [ ]:
df_inner = pd.merge(df_personal, df_research, on='個人番号', how='inner')
df_inner

左結合

In [ ]:
df_left = pd.merge(df_personal, df_research, on='個人番号', how='left')
df_left

右結合

In [ ]:
df_right = pd.merge(df_personal, df_research, on='個人番号', how='right')
df_right

外部結合

In [ ]:
df_outer = pd.merge(df_personal, df_research, on='個人番号', how='outer')
df_outer

**キーとなる列名がデータフレーム間で異なる場合：**

In [ ]:
import pandas as pd

# 個人番号
data1 = {
    '個人番号': [24561, 78943, 98276, 13456, 68745, 23689, 87654, 45123, 34902, 50389],
    '個人名': ['田中', '佐藤', '鈴木', '高橋', '伊藤', '山田', '中村', '小林', '加藤', '吉田'],
    '所属大学': ['東京大学', '京都大学', '早稲田大学', '一橋大学', '慶應義塾大学', '名古屋大学', '北海道大学', '神戸大学', '大阪大学', '九州大学'],
    '年齢': [28, 24, 26, 30, 27, 29, 25, 31, 23, 28],
    '科研費獲得額': [1000000, 500000, 300000, 800000, 600000, 400000, 700000, 200000, 900000, 350000]
}

df_personal_2 = pd.DataFrame(data1)

# パーソナル番号
data2 = {
    'パーソナル番号': [24561, 13456, 23689, 45123, 50389,99999],
    '研究エフォート率': [0.85, 0.92, 0.78, 0.75, 0.81,0.99],
    '職階': ['准教授', '助教授', '教授', '講師', '助手','教授']
}

df_research_2 = pd.DataFrame(data2)

従来の`on='個人番号'`ではKeyErrorが出力されます。

In [ ]:
# エラーが出るはず
df_inner_2 = pd.merge(df_personal_2, df_research_2, on='個人番号', how='inner')
df_inner_2

この場合、`on='個人番号'`の代わりに`left_on='個人番号',right_on='パーソナル番号'`を使用する必要があります。

In [ ]:
df_inner_2 = pd.merge(df_personal_2, df_research_2, left_on='個人番号',right_on='パーソナル番号',how='inner')
df_inner_2

**merge()後に各行の内訳を確認したい場合：**


`indicator=True`で内訳を確認できます。新たに作成される`_merge`列を参照ください。

In [ ]:
df_inner_2 = pd.merge(df_personal_2, df_research_2, left_on='個人番号',right_on='パーソナル番号',how='outer',indicator=True)
df_inner_2

#### 1. concat()
`concat()` 関数は、DataFrameを特定の軸に沿って結合します。デフォルトでは、DataFrameは縦（軸=0）に結合されます。

一般的に、concat()は形状が同一（つまり、列名が同一）のDataFrameやSeriesを結合する際に利用されます。

In [ ]:
df_concat = pd.concat([df_personal, df_research])
df_concat

軸を1に設定すると、DataFrameは横に結合されます。

しかし、共通の列が重複してしまうことに注意が必要です。

In [ ]:
df_concat_column = pd.concat([df_personal, df_research], axis=1)
df_concat_column

複数あるSeries(列)のconcatも可能です。

In [ ]:
# 縦にconcat
pd.concat([df_personal['所属大学'], df_research['職階'],df_research_2['パーソナル番号']])

In [ ]:
# 横に結合
pd.concat([df_personal['所属大学'], df_research['職階'],df_research_2['パーソナル番号']], axis=1)

#### 3. join()

`join()` 関数もDataFrameの結合に使用されますが、キーとしてインデックスを使用します。結合方法は `merge()` 関数と同じく `how` パラメータを使用して制御できます。


**インデックスの設定**

`個人番号`をインデックスに設定します。これは、`join()`がデフォルトでインデックスを基に結合を行うからです。

In [ ]:
df_personal_join = df_personal.set_index('個人番号')
df_research_join = df_research.set_index('個人番号')

In [ ]:
df_personal_join

In [ ]:
df_research_join

**データフレームの結合**

最後に、`join()`を使って2つのデータフレームを結合します。

ここでは、`how='left'`を指定して左結合を行っています。

これは、`df_personal`の全ての行を保持し、それに対応する`df_research`の行が存在する場合にその情報を結合するためです。

対応する行が`df_research`に存在しない場合、結合後の対応する列はNaN（値が存在しない）になります。

結合が完了したら、`df`には`df_personal`と`df_research`の両方の情報が含まれます。

In [ ]:
df_join = df_personal_join.join(df_research_join, how='left')
df_join

## ■ データの出力

### データのCSV出力

前処理が終了したデータは、将来的な使用のためにCSVファイルとして保存することが一般的です。

以下のコードは、前述のフィルタリングされたデータをCSVファイルとして出力します。


In [ ]:
df_output = df_personal_join.join(df_research_join, how='inner')
df_output = df_output[df_output['科研費獲得額'] > 500_000] # 500000以上のみ抽出
df_output

In [ ]:
# データをCSVファイルとして出力します
df_output.to_csv('output_data.csv', index=False)

## ■ オマケ：パイプラインについて

In [ ]:
## ファイルの取得
!wget -P . https://raw.githubusercontent.com/gmoriki/C4RA-Python-Tutorials/master/0609/kaken.nii.ac.jp_2023-06-07_10-57-32.csv

In [ ]:
df_kaken = pd.read_csv('./kaken.nii.ac.jp_2023-06-07_10-57-32.csv')

In [ ]:
# copy
df_kaken_1 = df_kaken.copy()
# こうやって書くと大変

# 欠損値を削除
df_kaken_1 = df_kaken_1.dropna(axis=1, how='any')

# 必要な列だけを選択
df_kaken_1 = df_kaken_1[['研究課題名','研究種目','総配分額']]

# 条件に合う行を選択
mask = (df_kaken_1['総配分額'] > df_kaken_1['総配分額'].mode().values[0]) & (df_kaken_1['研究種目'].str.contains('C'))
df_kaken_1 = df_kaken_1[mask]

# '総配分額'でソート
df_kaken_1 = df_kaken_1.sort_values(by='総配分額')

# インデックスをリセット
df_kaken_1 = df_kaken_1.reset_index(drop=True)
df_kaken_1

In [ ]:
# copy
df_kaken_2 = df_kaken.copy()
# 複数の処理を続けて書くことができます(パイプライン)
df_kaken_2 = df_kaken_2.dropna(axis=1, how='any')[['研究課題名','研究種目','総配分額']][(df_kaken_2['総配分額'] > df_kaken_2['総配分額'].mode().values[0]) & (df_kaken_2['研究種目'].str.contains('C'))].sort_values(by='総配分額').reset_index(drop=True)
df_kaken_2

In [ ]:
# copy
df_kaken_3 = df_kaken.copy()

# 見やすい例1
df_kaken_3 = (df_kaken_3.dropna(axis=1, how='any') \
            [['研究課題名','研究種目','総配分額']] \
            [(df_kaken_3['総配分額'] > df_kaken_3['総配分額'].mode().values[0]) & (df_kaken_3['研究種目'].str.contains('C'))] \
            .sort_values(by='総配分額') \
            .reset_index(drop=True)
            ) 
df_kaken_3

In [ ]:
# copy
df_kaken_4 = df_kaken.copy()

# 見やすい例2
def drop_na_cols(df):
    return df.dropna(axis=1, how='any')

def filter_rows(df):
    return df[(df['総配分額'] > df['総配分額'].mode()[0]) & (df['研究種目'].str.contains('C'))]

def select_columns(df):
    return df[['研究課題名','研究種目','総配分額']]

def sort_values(df):
    return df.sort_values(by='総配分額')

def reset_idx(df):
    return df.reset_index(drop=True)

df_kaken_4 = (df_kaken_4
            .pipe(drop_na_cols)
            .pipe(filter_rows)
            .pipe(select_columns)
            .pipe(sort_values)
            .pipe(reset_idx)
           )
df_kaken_4

In [ ]:
# copy
df_kaken_5 = df_kaken.copy()

# 見やすい例3
def process_dataframe(df):
    df = df.dropna(axis=1, how='any')
    df = df[(df['総配分額'] > df['総配分額'].mode()[0]) & (df['研究種目'].str.contains('C'))]
    df = df[['研究課題名','研究種目','総配分額']]
    df = df.sort_values(by='総配分額')
    df = df.reset_index(drop=True)
    return df

df_kaken_5 = df_kaken_5.pipe(process_dataframe)
df_kaken_5